# Run Simulations

## Import relevant libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

from configparser import ConfigParser

from typing import Callable

import gym
import gym_anytrading

from gym_anytrading.envs import CryptoEnvLogBLSH

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO, DQN
from stable_baselines3.common.evaluation import evaluate_policy

import tensorflow as tf

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os

import multiprocessing

import quantstats as qs

In [2]:
# gpu = len(tf.config.list_physical_devices('GPU'))>0
# print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [4]:
%matplotlib inline

## Get values from the config file

In [5]:
configur = ConfigParser()
config_file_name = "config_00a_mix.ini"
print (configur.read(os.path.join('config_files', config_file_name)))

['config_files/config_00a_mix.ini']


In [6]:
config_file_name_without_extension = config_file_name.replace(".ini", "")

ccy = configur.get('data', 'ccy')
data_frequency_train = configur.get('data', 'data_frequency_train')
data_frequency_val = configur.get('data', 'data_frequency_val')

window_size = configur.getint('environment', 'window_size')
start_date = configur.get('environment', 'start_date')
mid_date = configur.get('environment', 'mid_date')
end_date = configur.get('environment', 'end_date')
list_features = configur.get('environment', 'features').split(',')
target = configur.get('environment', 'target')

num_of_simulations = configur.getint('simulation', 'num_of_simulations')

In [7]:
def get_and_process_data(ccy, data_frequency):
    try:
        
        # Get the data
        filename_data = 'binance_' + ccy + "_" + data_frequency + '_from_2017_01_01_to_2022_12_31_candlesticks_signals_processed_technical_indicators_and_crypto_index_log.csv'
        fullpath_data = os.path.join('../../data/50_log/', filename_data)
        df = pd.read_csv(fullpath_data)

        # Converting Date Column to DateTime Type
        df['Date'] = pd.to_datetime(df['formatted_open_time'])

        # Setting the column as index
        df.set_index('Date', inplace=True)

        return df

    except Exception as e:
        print(e)

In [8]:
def my_process_data(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, target].to_numpy()[start:end]
    signal_features = env.df.loc[:, list_features].to_numpy()[start:end]
    dates = env.df.index.to_numpy()[start:end]
    return prices, signal_features, dates

class MyEnv(CryptoEnvLogBLSH):
    _process_data = my_process_data

In [9]:
df_train = get_and_process_data(ccy, data_frequency_train)
df_val = get_and_process_data(ccy, data_frequency_val)

In [10]:
print(f'df_train.shape: {df_train.shape}')
print(f'df_val.shape: {df_val.shape}')

df_train.shape: (46860, 94)
df_val.shape: (1839, 94)


In [11]:
start_date_id_train = int(df_train.index.get_loc(start_date))
mid_date_id_train = int(df_train.index.get_loc(mid_date))
mid_date_id_val = int(df_val.index.get_loc(mid_date))
end_date_id_val = int(df_val.index.get_loc(end_date))

In [12]:
#list_models = ['A2C', 'PPO', 'DQN', 'RANDOM']
list_models = ['A2C']

path_tensorboard = os.path.join("tensorboard", config_file_name.replace(".ini", ""))

df_sim_results = pd.DataFrame()

#setting up our environment for training 
env_maker = lambda: MyEnv(df=df_train, frame_bound=(start_date_id_train,mid_date_id_train), window_size=window_size)
env = DummyVecEnv([env_maker])

sim_id = 1

for model_name in list_models:

    if(model_name == 'A2C'):
        model = A2C('MlpPolicy', env, verbose=0, tensorboard_log=path_tensorboard) 
    elif(model_name == 'PPO'):
        model = PPO('MlpPolicy', env, batch_size=1024, verbose=0, tensorboard_log=path_tensorboard)
    elif(model_name == 'DQN'):
        model = DQN('MlpPolicy', env, batch_size=1024, verbose=0, tensorboard_log=path_tensorboard)

    #setting the learning timesteps
    model.learn(total_timesteps=(mid_date_id_train - start_date_id_train))
    
    # Export the model components
    
    if(model_name == 'A2C'):
        model.save(os.path.join('save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
        model.policy.save(os.path.join('save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))
    elif(model_name == 'PPO'):
        model.save(os.path.join('save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
        model.policy.save(os.path.join('save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))
    elif(model_name == 'DQN'):
        model.save(os.path.join('save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
        model.save_replay_buffer(os.path.join('save_models_components', 'replay_buffer', 'replay_buffer_' + model_name + "_" + config_file_name_without_extension))
        model.policy.save(os.path.join('save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))

    df_sim_results_temp = pd.DataFrame(columns=['total_reward_cash', 'total_profit_percentage', 'fmt_total_profit_percentage', 'num_of_trades'])

    list_sim_id = []

    for i in range(num_of_simulations):   
        
        list_sim_id.append(sim_id)

        # Check the progress
        if(i % 10 == 0):
            print(i)

        env = MyEnv(df=df_val, frame_bound=(mid_date_id_val,end_date_id_val), window_size=window_size, log=True)

        #Setting up the Agent Environment
        obs = env.reset()
        while True: 
            obs = obs[np.newaxis, ...]
            if(model_name == 'RANDOM'):
                action = env.action_space.sample()
            else:
                action, _states = model.predict(obs)
            obs, rewards, done, info = env.step(action)
            if done:
                df_sim_results_temp = df_sim_results_temp.append(info, ignore_index=True, sort=False)
                break

        # Export robot actions plot
        fig = env.render_all()
        fig.write_html(os.path.join('results', 'plots', 'robot_actions', 'robot_actions_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + '.html'))

        # Export quantstats
        qs.extend_pandas()
        net_worth = pd.Series(env.history['total_profit_percentage'], index=df_val.index[mid_date_id_val+1:end_date_id_val])
        returns = net_worth.pct_change().iloc[1:]
        qs.reports.html(returns, output='quantstats_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + '.html', title=config_file_name_without_extension + '_sim_id_' + str(sim_id), download_filename=os.path.join('results', 'plots', 'quantstats', 'quantstats_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + '.html'))

        sim_id = sim_id + 1
        
    df_sim_results_temp['sim_id'] = list_sim_id
    df_sim_results_temp['ccy'] = ccy
    df_sim_results_temp['data_frequency_train'] = data_frequency_train
    df_sim_results_temp['data_frequency_val'] = data_frequency_val
    df_sim_results_temp['window_size'] = window_size
    df_sim_results_temp['start_date'] = start_date
    df_sim_results_temp['mid_date'] = mid_date
    df_sim_results_temp['end_date'] = end_date
    df_sim_results_temp['list_features'] = str(list_features)
    df_sim_results_temp['model_type'] = model_name 
    df_sim_results_temp['num_of_simulations'] = num_of_simulations   

    df_sim_results = pd.concat([df_sim_results, df_sim_results_temp], axis=0)

    

0


ValueError: Length of values (92) does not match length of index (91)

## Results Analysis

### Pre-process results

In [ ]:
df_sim_results = df_sim_results.reset_index(drop=True)

In [ ]:
df_sim_results = df_sim_results.drop_duplicates()

### Display the results

In [ ]:
x = df_sim_results[(df_sim_results['model_type'] == 'A2C')]
x[x.total_reward_cash == x.total_reward_cash.max()]

### Plots

In [ ]:
sns.set(rc={"figure.figsize":(15, 10)})

str_title = f'ccy:{ccy} | data_frequency_train:{data_frequency_train} | data_frequency_val:{data_frequency_val} | window_size:{window_size} | start_run_date:{mid_date} | end_run_date:{end_date} | lenght list_features:{len(list_features)} | num_of_simulations:{num_of_simulations}'

file_export_plot_sim_results = config_file_name.replace(".ini", ".png")

In [ ]:
sns.boxplot(data=df_sim_results, x="fmt_total_profit_percentage", y="model_type").set(title=str_title)
plt.savefig(os.path.join('results', 'plots', 'total_profit_percentage', 'box_plot_' + file_export_plot_sim_results))


In [ ]:
sns.histplot(data=df_sim_results, x="fmt_total_profit_percentage", hue="model_type", kde=True).set(title=str_title)
plt.savefig(os.path.join('results', 'plots', 'total_profit_percentage', 'hist_plot_' + file_export_plot_sim_results))

In [ ]:
sns.histplot(data=df_sim_results, x="num_of_trades", hue="model_type", kde=True).set(title=str_title)
plt.savefig(os.path.join('results', 'plots', 'num_of_trades', 'hist_plot_' + file_export_plot_sim_results))

## Export results

In [ ]:
file_export_sim_results = config_file_name.replace(".ini", ".csv")
fullpath_export_sim_results = os.path.join('results', 'data', 'sim_results_' + file_export_sim_results)

In [ ]:
df_sim_results.to_csv(fullpath_export_sim_results)